In [1]:
# Import requests library
import requests
import getpass
rapidAPIkey = getpass.getpass()

········


In [ ]:
#First work with searching by airport

In [57]:
def flight_prices(departure, arrival, date, return_date):
    departure_code = departure
    arrival_code = arrival
    url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/{departure_code}/{arrival_code}/{date}"
    params = {"inboundpartialdate":{return_date}}
    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': rapidAPIkey}
    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [58]:
response = flight_prices("BCN-sky", "GRU-sky", "2021-09-28", "2021-12-02")
response

{'Quotes': [{'QuoteId': 1,
   'MinPrice': 294,
   'Direct': False,
   'OutboundLeg': {'CarrierIds': [838],
    'OriginId': 42414,
    'DestinationId': 54661,
    'DepartureDate': '2021-09-28T00:00:00'},
   'QuoteDateTime': '2021-09-12T16:02:00'}],
 'Carriers': [{'CarrierId': 838, 'Name': 'Air France'}],
 'Places': [{'Name': 'Barcelona',
   'Type': 'Station',
   'PlaceId': 42414,
   'IataCode': 'BCN',
   'SkyscannerCode': 'BCN',
   'CityName': 'Barcelona',
   'CityId': 'BARC',
   'CountryName': 'Spain'},
  {'Name': 'Sao Paulo Guarulhos',
   'Type': 'Station',
   'PlaceId': 54661,
   'IataCode': 'GRU',
   'SkyscannerCode': 'GRU',
   'CityName': 'Sao Paulo',
   'CityId': 'SAOA',
   'CountryName': 'Brazil'}],
 'Currencies': [{'Code': 'USD',
   'Symbol': '$',
   'ThousandsSeparator': ',',
   'DecimalSeparator': '.',
   'SymbolOnLeft': True,
   'SpaceBetweenAmountAndSymbol': False,
   'RoundingCoefficient': 0,
   'DecimalDigits': 2}]}

In [60]:
# Import libraries
import pandas as pd
import json

# Normalize the response

pd.json_normalize(response)
# If this doesn't work, try to import json_normalize directly from pandas.io.json

,Quotes,Carriers,Places,Currencies
0,"[{'QuoteId': 1, 'MinPrice': 294, 'Direct': Fal...","[{'CarrierId': 838, 'Name': 'Air France'}]","[{'Name': 'Barcelona', 'Type': 'Station', 'Pla...","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."


In [61]:
quotes = pd.DataFrame(pd.json_normalize(response)["Quotes"][0])
carriers = pd.DataFrame(pd.json_normalize(response)["Carriers"][0])
places = pd.DataFrame(pd.json_normalize(response)["Places"][0])
currencies = pd.DataFrame(pd.json_normalize(response)["Currencies"][0])

In [78]:
flights = pd.DataFrame(pd.DataFrame(pd.json_normalize(response)["Quotes"][0])["MinPrice"])

In [79]:
flights

,MinPrice
0,294


In [81]:
flights2 = pd.concat([carriers, places, currencies, flights], axis=1)
flights2 = flights2[["Name", "CityName", "CountryName","MinPrice"]]
flights2

,Name,Name,CityName,CountryName,MinPrice
0,Air France,Barcelona,Barcelona,Spain,294.0
1,NaN,Sao Paulo Guarulhos,Sao Paulo,Brazil,NaN


In [ ]:
#working with city, but discover is not possible in the api search for a list of places id.
#let the code here to see my idea

In [51]:
def city_code(city_name):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/US/USD/en-US/"

    params = {"query": city_name}

    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
                      'x-rapidapi-key': rapidAPIkey}

    response = requests.get(url, headers = headers, params = params).json()
    
    count = len(response["Places"])
    codes = []
    for i in range(count):
        codes.append(response["Places"][i]["PlaceId"])
    
    return codes

def flight_prices(departure, arrival, date, return_date):
    departure_code = city_code(departure)
    arrival_code = city_code(arrival)
    url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/{departure_code}/{arrival_code}/{date}"
    params = {"inboundpartialdate":{return_date}}
    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': rapidAPIkey}
    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [52]:
response = flight_prices("Barcelona", "São Paulo", "2021-09-28", "2021-12-02")
response

{'errors': ['path param destination  is an invalid destination',
  'path param origin  is an invalid origin']}